In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
from mysql.connector import Error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configurações do banco de dados
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "12344",
    "database": "mercado_imobiliario"
}

# Função para conectar ao banco de dados
def conectar_banco():
    try:
        conexao = mysql.connector.connect(**DB_CONFIG)
        return conexao
    except Error as e:
        print(f"Erro ao conectar ao banco: {e}")
        return None

# Função para listar tabelas disponíveis
def listar_tabelas():
    conexao = conectar_banco()
    if conexao:
        cursor = conexao.cursor()
        cursor.execute("SHOW TABLES;")
        tabelas = [tabela[0] for tabela in cursor.fetchall()]
        conexao.close()
        return tabelas
    return []

# Função para carregar os dados do banco
def carregar_dados():
    tabelas = listar_tabelas()
    if not tabelas:
        print("Nenhuma tabela encontrada no banco de dados.")
        return None
    
    tabela = tabelas[0]  # Carregar a primeira tabela disponível
    print(f"Carregando dados da tabela: {tabela}")
    conexao = conectar_banco()
    if conexao:
        query = f"SELECT * FROM `{tabela}`;"
        df = pd.read_sql(query, conexao)
        conexao.close()
        return df
    else:
        return None

# Carregar os dados
df = carregar_dados()
if df is None or df.empty:
    print("Erro ao carregar dados do banco ou DataFrame vazio.")
else:
    # Ajustar nomes de colunas
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")
    
    # Selecionar colunas relevantes
    colunas_numericas = ['area', 'quartos', 'banheiros', 'latitude', 'longitude']
    colunas_categoricas = ['municipio', 'estado']
    target = 'preco'
    
    df = df.dropna(subset=[target])
    
    # Separar features e target
    X = df[colunas_numericas + colunas_categoricas]
    y = df[target]
    
    # Divisão em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Pipeline de transformação dos dados
    preprocessor = ColumnTransformer([
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
    ])
    
    # Modelo de Regressão Linear
    reg_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])
    
    # Treinar o modelo
    reg_pipeline.fit(X_train, y_train)
    
    # Previsões
    y_pred = reg_pipeline.predict(X_test)
    
    # Avaliação do modelo
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    
    # Plot das previsões
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)
    plt.xlabel("Preços reais")
    plt.ylabel("Previsões")
    plt.title("Regressão Linear: Preço dos Imóveis")
    plt.show()


Carregando dados da tabela: sample_submission


KeyError: ['preco']